In [1]:
!pip install python-telegram-bot
!pip install tensorflow
!pip install matplotlib
!pip install pillow

# Install necessary libraries and tools
!pip install -q tensorflow==2.10.0  # You can specify the version that fits your needs
!pip install -q protobuf  # Install protobuf

# Clone the TensorFlow models repository and modify visualization utils
!rm -rf ./models && git clone https://github.com/tensorflow/models.git
!sed -i "s#ImageFont.truetype('arial.ttf', 24)#ImageFont.truetype('arial.ttf', 50)#g" ./models/research/object_detection/utils/visualization_utils.py

# Install the required fonts
!apt-get install -y fonts-dejavu

# Compile Protobuf files and install Object Detection API
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!pip install -e .

# Verify installation
import os
import sys
sys.path.append('/content/models/research')
import object_detection

print("TensorFlow version:", tf.__version__)
print("Object Detection API installed successfully")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 26.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take 

NameError: name 'tf' is not defined

In [ ]:

#


In [28]:
import os
import json
import numpy as np
import tensorflow as tf
from PIL import Image
import aiohttp
import pandas as pd
import difflib
from google.colab import drive
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import label_map_util
from object_detection.protos import string_int_label_map_pb2
from google.protobuf import text_format
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
import nest_asyncio
nest_asyncio.apply()
# Constants
TOKEN = '7514049535:AAEvxdhSYYeMRwpQoxu1zic4OA5eFX-IrgQ'
threshold = 0.5
disposal_path = '/content/drive/MyDrive/final.csv'
model_path = '/content/drive/MyDrive/saved_modelTACO/ssd_mobilenet_v2_taco_2018_03_29.pb'
annotations_file = '/content/drive/MyDrive/Garbage Goober/annotations.json'

# Initialize asyncio
nest_asyncio.apply()

# Load and preprocess disposal locations data
df = pd.read_csv(disposal_path)
df['block'] = df['block'].fillna('').astype(str)
df['name'] = df['name'].fillna('').astype(str).str.lower()
df['combined'] = df['block'] + " " + df['name']

def find_similar_locations(user_input):
    user_input = user_input.lower()
    similarities = df['combined'].apply(lambda x: difflib.SequenceMatcher(None, x, user_input).ratio())
    df['similarity'] = similarities
    top_matches = df.sort_values(by='similarity', ascending=False).head(5)
    output_message = "You can go to these locations:\n"
    for _, row in top_matches.iterrows():
        output_message += f"~ {row['name']} block {row['block']}, postalcode: {row['postalcode']}, Proximity Score: {row['similarity']:.2f}\n"
    return output_message

# Recyclable categories
recyclable_ids = {4, 10, 12, 14, 15, 16, 18, 30, 31, 34, 35, 36, 37, 38, 39, 40, 41, 42, 49, 51, 52}
needs_to_be_clean_ids = {5, 6, 7, 8, 9, 11, 13, 17, 19, 21, 22, 24, 25, 27, 28, 29, 45, 46, 48, 50, 56, 57}
not_ids = {1, 2, 3, 20, 32, 33, 43, 44, 47, 58, 59, 54, 55, 53, 23, 26}

# Load and initialize TensorFlow model
def reconstruct(pb_path):
    if not os.path.isfile(pb_path):
        print(f"Error: {pb_path} not found")
        return None
    print("Reconstructing TensorFlow model")
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.compat.v1.GraphDef()
        with tf.io.gfile.GFile(pb_path, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    print("Success!")
    return detection_graph

def image2np(image):
    (w, h) = image.size
    return np.array(image.getdata()).reshape((h, w, 3)).astype(np.uint8)

def image2tensor(image):
    npim = image2np(image)
    return np.expand_dims(npim, axis=0)

def detect_objects(detection_graph, image_np):
    with detection_graph.as_default():
        with tf.compat.v1.Session(graph=detection_graph) as sess:
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded}
            )
    return boxes, scores, classes, num

def process_detection_results(boxes, scores, classes, num, category_index, threshold=threshold):
    detected_info = []
    num_detections = int(num[0])
    for i in range(num_detections):
        class_id = int(classes[0][i])
        score = scores[0][i]
        if score >= threshold:
            class_name = category_index.get(class_id, {'name': 'N/A'})['name']
            category = classify_category(class_id)
            detected_info.append({
                'class_id': class_id,
                'class_name': class_name,
                'score': score,
                'category': category
            })
    detected_classes = set(info['class_id'] for info in detected_info)
    return detected_info, len(detected_info), len(detected_classes)

def classify_category(class_id):
    if class_id in recyclable_ids:
        return 'Recyclable'
    elif class_id in needs_to_be_clean_ids:
        return 'Recyclable BUT needs to be cleaned'
    elif class_id in not_ids:
        return 'Non-recyclable'
    else:
        return 'Unknown'

def visualize_boxes(image_np, boxes, scores, classes, category_index, min_score_thresh=threshold):
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        instance_masks=None,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=min_score_thresh
    )
    return image_np

async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text('Send me a photo and I will process it!')

async def handle_photo(update: Update, context: CallbackContext):
    photo = update.message.photo[-1]
    file_id = photo.file_id
    file = await context.bot.get_file(file_id)

    async with aiohttp.ClientSession() as session:
        async with session.get(file.file_path) as resp:
            image_bytes = await resp.read()
            image_path = 'received_image.jpg'
            with open(image_path, 'wb') as f:
                f.write(image_bytes)

    image = Image.open(image_path)
    image_np = np.array(image)

    boxes, scores, classes, num = detect_objects(detection_graph, image_np)
    image_np = visualize_boxes(image_np, boxes, scores, classes, category_index)

    result_image = 'processed_image.jpg'
    Image.fromarray(image_np).save(result_image)
    await update.message.reply_photo(photo=open(result_image, 'rb'))

    detected_info, num_objects, num_classes = process_detection_results(boxes, scores, classes, num, category_index)
    result_text = f"Detected objects:\n"

    recyclable_detected = False
    for info in detected_info:
        result_text += f"Class: {info['class_name']}, Score: {info['score']:.2f}, Category: {info['category']}\n"
        if info['category'] in ['Recyclable', 'Recyclable BUT needs to be cleaned']:
            recyclable_detected = True
    result_text += f"\nTotal number of detected trash items: {num_objects}\n"
    result_text += f"Number of unique trash items detected: {num_classes}"

    await update.message.reply_text(result_text)

    if recyclable_detected:
        context.user_data['waiting_for_address'] = True
        await update.message.reply_text("The detected item is recyclable. Please provide your address so I can find nearby recycling locations.")
    else:
        context.user_data['waiting_for_address'] = False

async def handle_message(update: Update, context: CallbackContext):
    if context.user_data.get('waiting_for_address', False):
        user_address = update.message.text
        locations_message = find_similar_locations(user_address)
        await update.message.reply_text(locations_message)
        context.user_data['waiting_for_address'] = False
    else:
        await update.message.reply_text("Please send a photo first to process.")

# Load the model and category index
drive.mount('/content/drive')
detection_graph = reconstruct(model_path)

NCLASSES = 60

with open(annotations_file) as json_file:
    data = json.load(json_file)

categories = data['categories']
labelmap = string_int_label_map_pb2.StringIntLabelMap()
for idx, category in enumerate(categories):
    item = labelmap.item.add()
    item.id = int(category['id'])
    item.name = category['name']
category_index = label_map_util.create_category_index(categories)
# for category_id, category_info in category_index.items():
#     print(f"ID: {category_id}, Name: {category_info['name']}")
# Initialize the Telegram bot
app = Application.builder().token(TOKEN).build()

app.add_handler(CommandHandler("start", start))
app.add_handler(MessageHandler(filters.PHOTO, handle_photo))
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reconstructing TensorFlow model
Success!
ID: 0, Name: Aluminium foil
ID: 1, Name: Battery
ID: 2, Name: Aluminium blister pack
ID: 3, Name: Carded blister pack
ID: 4, Name: Other plastic bottle
ID: 5, Name: Clear plastic bottle
ID: 6, Name: Glass bottle
ID: 7, Name: Plastic bottle cap
ID: 8, Name: Metal bottle cap
ID: 9, Name: Broken glass
ID: 10, Name: Food Can
ID: 11, Name: Aerosol
ID: 12, Name: Drink can
ID: 13, Name: Toilet tube
ID: 14, Name: Other carton
ID: 15, Name: Egg carton
ID: 16, Name: Drink carton
ID: 17, Name: Corrugated carton
ID: 18, Name: Meal carton
ID: 19, Name: Pizza box
ID: 20, Name: Paper cup
ID: 21, Name: Disposable plastic cup
ID: 22, Name: Foam cup
ID: 23, Name: Glass cup
ID: 24, Name: Other plastic cup
ID: 25, Name: Food waste
ID: 26, Name: Glass jar
ID: 27, Name: Plastic lid
ID: 28, Name: Metal lid
ID: 29, Name: Other plastic
ID: 30,

In [29]:
app.run_polling()

RuntimeError: Cannot close a running event loop

In [ ]:
!pip show python-telegram-bot

Name: python-telegram-bot
Version: 21.4
Summary: We have made you a wrapper you can't refuse
Home-page: https://python-telegram-bot.org
Author: 
Author-email: Leandro Toledo <devs@python-telegram-bot.org>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: httpx
Required-by: 
